## Investigating the potential benefits of optimization with other statistics 

* Goal as always is to obtain more robust solutions.
* What's so special about the mean?
* Look into percentiles.
* significance of tail contributions.
* Higher order moments?


In [3]:
import numpy as np
import matplotlib.pyplot as plt

D = 126.4          # rotor diameter

# x = np.array([0., 3.*D, 6.*D, 0., 3.*D, 6.*D, 0., 3.*D, 6.*D])            # x and y locations
# y = np.array([0., 0., 0., 3.*D, 3.*D, 3.*D, 6.*D, 6.*D, 6.*D])

x = [0., 3.*D, 3.*D, 0.]            # x and y locations
y = [0., 0., 3.*D, 3.*D]


In [5]:
import numpy as np

"""the jensen wake model"""

def Jensen_wake(x,y, U):
    
    k = 0.075      # wake decay constant
    D = 126.4      # rotor diameter
    Ct = 0.8       # thrust coefficient
    
    s = x/D        # relative distance downwind 
    
    Dw = D* (1. + 2.*k*s)               # wake diameter
       
    U_hub = U*(1. - (1. - np.sqrt(1. - Ct))/(1. + s*k*s)**2)          # hub velocity
    
    return U_hub

In [6]:
""" Power calculation """


def jensen_power(x, y, U):
    
    rho = 1.1716                   # air density   
    a = 1. / 3.                    # axial induction
    Cp = 4.*a*(1-a)**2.            #power coefficient
    
    D = 126.4
    
    r0 = D/2.

    A = np.pi*r0**2        #  rotor swept area
    
    U_hub1 = np.zeros(len(x))
    Power = np.zeros(len(x))
    
    
    U_hub1[0] = U
    U_hub1[1] = Jensen_wake((x[1]-x[0]), (y[1]-y[0]), U)
    U_hub1[2] = Jensen_wake((x[2]-x[3]), (y[2]-y[3]), U)
    U_hub1[3] = U
    
    # Calculate power from each turbine
    for i in range(len(U_hub1)):
        
        Power[i] = 0.5*rho*Cp*A*U_hub1[i]**3
           
    return Power

U = 8.

print "Power:", jensen_power(x,y, U), "W"


Power: [ 2230280.95623081  1154678.1330182   1154678.1330182   2230280.95623081] W


In [12]:
import chaospy  as cp

dist = cp.Weibull(1.8, 12.55, 0.) # wind speed distribution

U_inf = dist.sample(120000, "M")

solves = np.zeros((len(U_inf), len(x)))
mu_mc = np.zeros(len(U_inf))

for i in range(len(solves)):
    solves[i,:] = jensen_power(x,y, U_inf[i])# for s in U_inf]
    mu_mc[i] = np.mean(solves[i,:])

# print solves

AEP_mc1 = 8760*np.mean(mu_mc)/1e9, "GWh"
AEP_mc2 = 8760*np.percentile(mu_mc, 90)/1e9, "GWh"

print AEP_mc1
print AEP_mc2

(86.102098875051453, 'GWh')
(229.80560101567445, 'GWh')


In [ ]:
# Quadrature

order = 5

poly = cp.orth_ttr(order, dist)
nodes, weights = cp.generate_quadrature(order+1, dist, rule="G")

solves1 = [jensen_power(x, y, s) for s in nodes.T]
P_hat1 = cp.fit_quadrature(poly, nodes, weights, solves1)

mu_quad = cp.E(P_hat1, dist)     # mean from the quadrature method

AEP_quad = 8760*np.mean(mu_quad)/1e9

print AEP_quad,":,GWh"